In [ ]:
!pip install datasets
!pip install rouge_score

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
#Saving into log (Excel file)
import openpyxl 
def SaveToExperimentLog(Experiments_file, LogEntry, data):
    book = openpyxl.load_workbook(Experiments_file)
    writer = pd.ExcelWriter(Experiments_file, engine='openpyxl') 
    writer.book = book

    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    data.to_excel(writer, LogEntry[0:29],index=False)

    writer.save()
    writer.close()

In [ ]:
#Experiment
#Experiments log file 
Experiments_file='/content/drive/MyDrive/Colab Notebooks/Projects/eva/ExperimentLogs/Headlines.xlsx'
Experiments_tab='Validation'
Prediction_tab='Predictions'

source_field='message'
target_field='Topic'

In [ ]:
test_records = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Prediction_tab)

In [ ]:
import datasets
from datasets import load_metric
metric = load_metric("rouge")

def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [ ]:
test_records.columns

Index(['message', 'Topic', 'Bl_-1', 'Bl_0', 'FirstModel_Headline_1',
       'Model2_Headline_2', 'Model3_Headline_3', 'Model4_Headline_4',
       'Model5_Headline_5', 'Model6_Headline_6', 'Model7_Headline_7',
       'Model7_Headline_8', 'Model8_Headline_9', 'Model8_Headline_10',
       'IlyaGusev_Headline_11', 'Model9_Headline_12', 'Model10_Headline_13',
       'Model10_Headline_14', 'Model11_Headline_15', 'Model12_Headline_16',
       'Model13_Headline_17', 'Model14_Headline_18', 'Model15_Headline_19',
       'Model11_Headline_20'],
      dtype='object')

In [ ]:
model_prediction_columns=['Bl_-1', 'Bl_0', 'FirstModel_Headline_1',
       'Model2_Headline_2', 'Model3_Headline_3', 'Model4_Headline_4',
       'Model5_Headline_5', 'Model6_Headline_6', 'Model7_Headline_7',
       'Model7_Headline_8', 'Model8_Headline_9', 'Model8_Headline_10',
       'IlyaGusev_Headline_11', 'Model9_Headline_12', 'Model10_Headline_13',
       'Model10_Headline_14', 'Model11_Headline_15', 'Model12_Headline_16',
       'Model13_Headline_17', 'Model14_Headline_18', 'Model15_Headline_19','Model11_Headline_20']

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def bleu1(row,reference_field,candidate_field):
  return sentence_bleu([row[reference_field].split()], row[candidate_field].split(), weights=(1, 0, 0, 0))

def bleu2(row,reference_field,candidate_field):
  return sentence_bleu([row[reference_field].split()], row[candidate_field].split(), weights=(0.5, 0.5, 0, 0))  

def bleu3(row,reference_field,candidate_field):
  return sentence_bleu([row[reference_field].split()], row[candidate_field].split(), weights=(0.33, 0.33, 0.33, 0))    

def bleu4(row,reference_field,candidate_field):
  return sentence_bleu([row[reference_field].split()], row[candidate_field].split(), weights=(0.25, 0.25, 0.25, 0.25))    

In [ ]:
for c in model_prediction_columns:
  print('Processing %s'%c)
  print('BLEU to headline')
  test_records['%s_t_bleu1'%c] = test_records.apply(lambda x: bleu1(x, target_field,c), axis=1)
  test_records['%s_t_bleu2'%c] = test_records.apply(lambda x: bleu2(x, target_field,c), axis=1)
  test_records['%s_t_bleu3'%c] = test_records.apply(lambda x: bleu3(x, target_field,c), axis=1)
  test_records['%s_t_bleu4'%c] = test_records.apply(lambda x: bleu4(x, target_field,c), axis=1)
  print('BLEU to message')
  test_records['%s_m_bleu1'%c] = test_records.apply(lambda x: bleu1(x, source_field,c), axis=1)
  test_records['%s_m_bleu2'%c] = test_records.apply(lambda x: bleu2(x, source_field,c), axis=1)
  test_records['%s_m_bleu3'%c] = test_records.apply(lambda x: bleu3(x, source_field,c), axis=1)
  test_records['%s_m_bleu4'%c] = test_records.apply(lambda x: bleu4(x, source_field,c), axis=1)  


In [ ]:
SaveToExperimentLog(Experiments_file, Prediction_tab, test_records)

In [ ]:
BLEU_test_records = pd.DataFrame()
for c in model_prediction_columns:
  r = test_records[['%s_t_bleu1'%c,'%s_t_bleu2'%c,'%s_t_bleu3'%c,'%s_t_bleu4'%c,'%s_m_bleu1'%c,'%s_m_bleu2'%c,'%s_m_bleu3'%c,'%s_m_bleu4'%c]].mean()
  BLEU_test_records=BLEU_test_records.append(pd.DataFrame(r.values).T,ignore_index=True)
BLEU_test_records.columns=['t_bleu1','t_bleu2','t_bleu3','t_bleu4','m_bleu1','m_bleu2','m_bleu3','m_bleu4']  

In [ ]:
len(BLEU_test_records)

21

In [ ]:
Experiment = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiments_tab)    

In [ ]:
#Experiment[Experiment['Run']!=20]

In [ ]:
Experiment=pd.concat([Experiment, BLEU_test_records], axis=1)

In [ ]:
SaveToExperimentLog(Experiments_file, Experiments_tab, Experiment)

In [ ]:
test_records['size'] = test_records[source_field].str.split().apply(len)

In [ ]:
from numpy.core.defchararray import find
from numpy.core.defchararray import replace

In [ ]:
a = np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.upper(test_records[source_field].values.astype(str)),'.',''),',',''),'!',''),'?',''),':',''),';','')
b = np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.upper(test_records[target_field].values.astype(str)),'.',''),',',''),'!',''),'?',''),':',''),';','')
test_records['Topic_pos']=find(a, b)

In [ ]:
test_records[[source_field, target_field, 'size','Topic_pos']].sort_values(['size'],ascending=True).head(10)

,message,Topic,size,Topic_pos
3334,........ по океану.........матрос.........обез...,помогите придумать буриме,3,-1
1759,Нужны умные москвичи или жители Подмосковья.Пр...,"Собираю команду ""Что? Где? Когда?""",6,-1
2056,Наткнулась на фото на одноклассниках:школа №15...,"1 сентября, после линейки.",7,-1
3645,"Подскажите, пжста, какой ваш форум местный сам...",Кто из Питера?,8,-1
1960,Форум Насикомых читаете? Давайте кого нибудь о...,А вы читаете Насиков?,8,-1
1023,"Колечко золотое на работе, знаю чье.Отдать или...",Нашла,8,-1
1838,Служат сыновья или мужья срочную службу? Что р...,У кого сейчас в армии..,8,-1
2636,Пропавшая дочь топ-менеджера «Лукойла» найдена...,Пропавшая дочь топ-менеджера «Лукойла» найдена...,8,0
653,"В Москве дождищща!Грозищща!Молнии сверкают, вс...",В Москве дождищща!,8,0
3274,Посоветуйте какие-нибудь студии в районе новог...,где научиться рисовать,8,-1


In [ ]:
def HeaderType(row,column):
  if row[column]==-1:
    return 'Abstractive'
  elif  row[column]<=row['size']/3:
    return 'From Start'
  elif  row[column]>row['size']/3 and row[column]<2*row['size']/3:
    return 'From Middle'   
  elif row[column]>=2*row['size']/3:
    return 'From End'

In [ ]:
test_records['Topic_Type'] = test_records.apply(lambda x: HeaderType(x, 'Topic_pos'), axis=1)

In [ ]:
a = np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.upper(test_records[source_field].values.astype(str)),'.',''),',',''),'!',''),'?',''),':',''),';','')
for c in model_prediction_columns:
  b = np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.upper(test_records[c].values.astype(str)),'.',''),',',''),'!',''),'?',''),':',''),';','')
  test_records['%s_pos'%c]=find(a, b)
  test_records['%s_Type'%c] = test_records.apply(lambda x: HeaderType(x, '%s_pos'%c), axis=1)

In [ ]:
SaveToExperimentLog(Experiments_file, Prediction_tab, test_records)

In [ ]:
stats_df = test_records.groupby(['Topic_Type']).size().reset_index(name='Topic Stats')
stats_df.columns = ['Type','Topic']
stats_df

,Type,Topic
0,Abstractive,3716
1,From End,150
2,From Middle,34
3,From Start,199


In [ ]:
for c in model_prediction_columns:
  df = test_records.groupby(['%s_Type'%c]).size().reset_index(name=c)
  df.columns = ['Type',c]
  stats_df = pd.merge(stats_df,df, left_on='Type', right_on='Type', how='left' )

In [ ]:
stats_df

,Type,Topic,Bl_-1,Bl_0,FirstModel_Headline_1,Model2_Headline_2,Model3_Headline_3,Model4_Headline_4,Model5_Headline_5,Model6_Headline_6,...,IlyaGusev_Headline_11,Model9_Headline_12,Model10_Headline_13,Model10_Headline_14,Model11_Headline_15,Model12_Headline_16,Model13_Headline_17,Model14_Headline_18,Model15_Headline_19,Model11_Headline_20
0,Abstractive,3716,4077.0,1954,2346,2455,2238,2551,2622,2591,...,2578,2619,2555,2485,2739,2608,2504,2511,2505,2739
1,From End,150,5.0,904,729,723,824,666,649,670,...,596,656,666,699,581,631,693,682,642,581
2,From Middle,34,NaN,196,168,179,179,160,172,161,...,126,153,172,188,145,167,171,185,175,145
3,From Start,199,17.0,1045,856,742,858,722,656,677,...,799,671,706,727,634,693,731,721,777,634


In [ ]:
SaveToExperimentLog(Experiments_file, 'Stats', stats_df)

In [ ]:
Total = len(test_records)

In [ ]:
stats_df[[target_field] + model_prediction_columns] = stats_df[[target_field] + model_prediction_columns].div(Total, axis=0).multiply(100)

In [ ]:
SaveToExperimentLog(Experiments_file, 'Stats Percent', stats_df)

In [ ]:
stats_df_t = stats_df.T
stats_df_t.reset_index(inplace=True)
stats_df_t.columns

Index(['index', 0, 1, 2, 3], dtype='object')

In [ ]:
stats_df_t.columns=['Model_Run', 'Abstractive','From End','From Middle','From Start']
stats_df_t=stats_df_t[stats_df_t['Model_Run']!='Type']
#stats_df_t=stats_df_t[stats_df_t['Model_Run']!='Topic']
stats_df_t

,Model_Run,Abstractive,From End,From Middle,From Start
1,Topic,90.656258,3.659429,0.829471,4.854843
2,Bl_-1,99.463284,0.121981,NaN,0.414735
3,Bl_0,47.670163,22.05416,4.781654,25.494023
4,FirstModel_Headline_1,57.233472,17.784826,4.098561,20.883142
5,Model2_Headline_2,59.892657,17.638448,4.366919,18.101976
6,Model3_Headline_3,54.598683,20.102464,4.366919,20.931935
7,Model4_Headline_4,62.234691,16.247865,3.903391,17.614052
8,Model5_Headline_5,63.966821,15.83313,4.196145,16.003903
9,Model6_Headline_6,63.210539,16.34545,3.927787,16.516223
10,Model7_Headline_7,62.62503,16.174677,4.269334,16.930959


In [ ]:
SaveToExperimentLog(Experiments_file, 'StatsT', stats_df_t)